In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import sys
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
import time
from collections import Counter
tqdm.pandas(tqdm_notebook)
import lightgbm as lgb
import gc
import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
pd.set_option('display.float_format', '{:.5f}'.format)
SEED = 51
np.random.seed(SEED)
%matplotlib inline

In [2]:
train = dd.read_csv('../input/train_final.csv')
test = dd.read_csv('../input/test_final.csv')
train = train.compute()
test = test.compute()
train = train.reset_index(drop=False)
test = test.reset_index(drop=False)

In [3]:
print(train.shape,test.shape)

(201917, 445) (123623, 444)


In [4]:
corr_ = ['new_month_diff_mean', 'h_month_diff_mean', 'a_month_diff_mean', 'month_diff_mean', 'hist_month_diff_mean', 'h_purchase_amount_max', 'h_num_most_common_category_3', 'card_id_total', 'h_purchase_amount_std', 'h_purchase_amount_mean', 'h_purchase_amount_sum']

In [5]:
train.drop(corr_,axis=1,inplace=True)
test.drop(corr_,axis=1,inplace=True)

In [6]:
#k = train.corr()

In [7]:
y = train['target']
del train['target']

In [8]:
train.head()

,index,card_id,feature_1,feature_2,feature_3,first_active_month,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_month_nunique,hist_month_mean,hist_month_min,hist_month_max,hist_hour_nunique,hist_hour_mean,hist_hour_min,hist_hour_max,hist_weekofyear_nunique,hist_weekofyear_mean,hist_weekofyear_min,hist_weekofyear_max,hist_weekday_mean,hist_day_nunique,hist_day_mean,hist_day_min,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_skew,hist_installments_sum,hist_installments_max,hist_installments_mean,hist_installments_var,hist_installments_skew,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_skew,hist_month_diff_max,hist_month_diff_min,hist_month_diff_var,hist_month_diff_skew,hist_authorized_flag_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_mean,hist_category_3_mean,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,new_month_mean,new_month_min,new_month_max,new_hour_nunique,new_hour_mean,new_hour_min,new_hour_max,new_weekofyear_nunique,new_weekofyear_mean,new_weekofyear_min,new_weekofyear_max,new_weekday_mean,new_weekday_min,new_weekday_max,new_day_nunique,new_day_mean,new_day_min,new_day_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_skew,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_installments_skew,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_month_lag_var,new_month_lag_skew,new_month_diff_var,new_month_diff_skew,new_weekend_mean,new_category_1_mean,new_category_2_mean,new_category_3_mean,new_price_mean,new_price_max,new_price_min,new_price_var,new_Christmas_Day_2017_mean,new_Children_day_2017_mean,new_Black_Friday_2017_mean,new_Mothers_Day_2018_mean,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,new_purchase_date_uptomin,hist_first_buy,hist_last_buy,new_first_buy,new_last_buy,card_id_cnt_ratio,purchase_amount_total,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_ratio,month_diff_ratio,month_lag_mean,month_lag_max,month_lag_min,category_1_mean,installments_total,installments_mean,installments_max,installments_ratio,price_total,price_mean,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio,active_year,active_month,h_af_sum,h_af_min,h_af_max,h_af_std,h_af_mean,h_af_median,h_how_many_city_visited,h_most_visited_city,h_num_most_visited_city,h_second_most_visited_city,h_num_second_most_visited_city,h_third_most_visited_city,h_num_third_most_visited_city,h_most_common_category_1,h_num_most_common_category_1,

In [9]:
"""corr_list = []
for i in range(len(k)):
    for j in range(len(k)):
        if (k.iloc[i,j] >= 0.99) & (i != j):
            corr_list.append([k.index[i],k.columns[j]])"""

'corr_list = []\nfor i in range(len(k)):\n    for j in range(len(k)):\n        if (k.iloc[i,j] >= 0.99) & (i != j):\n            corr_list.append([k.index[i],k.columns[j]])'

In [10]:
"""used_ = []
for i in corr_list:
    for j in i:
        used_.append(j)"""

'used_ = []\nfor i in corr_list:\n    for j in i:\n        used_.append(j)'

In [11]:
#len(set(used_))

In [12]:
#list(pd.Series(used_).value_counts().head(11).index)

In [13]:
stratified = False

if stratified:
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=SEED)
else:
    folds = KFold(n_splits= 5, shuffle=True, random_state=SEED)

# Create arrays and dataframes to store results
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()

In [14]:
temp = train.dtypes.reset_index(drop=False)
temp.columns = ['column','type']
for i in [x for x in list(temp[temp['type'] == 'object']['column']) if x not in ['card_id', 'first_active_month']]:
    le = preprocessing.LabelEncoder()
    le.fit(list(train[i].values) + list(test[i].values ) )
    train[i] = le.transform( list(train[i].values) )
    test[i] = le.transform( list(test[i].values) )

In [15]:
feats = [c for c in train.columns if c not in ['card_id', 'first_active_month'] ]

In [16]:
# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [17]:
%%time
# k-fold
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
    train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
    valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

    # set data structure
    lgb_train = lgb.Dataset(train_x,
                            label=train_y,
                            free_raw_data=False)
    lgb_test = lgb.Dataset(valid_x,
                           label=valid_y,
                           free_raw_data=False)

    # params optimized by optuna
    params = {'num_leaves': 111,
         'min_data_in_leaf': 149, 
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.02,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083 ,
         "metric": 'rmse',
         "lambda_l1": 0.2634,
         "random_state": SEED,
         "verbosity": -1}

    reg = lgb.train(
                    params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_test],
                    valid_names=['train', 'test'],
                    num_boost_round=10000,
                    early_stopping_rounds= 200,
                    verbose_eval=200
                    )

    oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
    sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    #fold_rmse = rmse(valid_y, oof_preds[valid_idx])
    print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
    del reg, train_x, train_y, valid_x, valid_y
    gc.collect()
print("Average RMSE = {}".format(rmse(y,oof_preds)))

Training until validation scores don't improve for 200 rounds.
[200]	train's rmse: 3.45398	test's rmse: 3.72949
[400]	train's rmse: 3.34687	test's rmse: 3.72344
[600]	train's rmse: 3.26688	test's rmse: 3.72205
Early stopping, best iteration is:
[556]	train's rmse: 3.28292	test's rmse: 3.72154
Fold  1 RMSE : 3.721539
Training until validation scores don't improve for 200 rounds.
[200]	train's rmse: 3.48792	test's rmse: 3.5832
[400]	train's rmse: 3.37994	test's rmse: 3.58112
Early stopping, best iteration is:
[313]	train's rmse: 3.42229	test's rmse: 3.57982
Fold  2 RMSE : 3.579820
Training until validation scores don't improve for 200 rounds.
[200]	train's rmse: 3.45305	test's rmse: 3.73009
[400]	train's rmse: 3.35326	test's rmse: 3.72811
[600]	train's rmse: 3.26711	test's rmse: 3.72938
Early stopping, best iteration is:
[429]	train's rmse: 3.33824	test's rmse: 3.72754
Fold  3 RMSE : 3.727541
Training until validation scores don't improve for 200 rounds.
[200]	train's rmse: 3.48029	test'

In [18]:
%%time
#num_leaves
folds_dict = {}
for i in [80,85,90,95,100]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': changed_param,
             'min_data_in_leaf': 149, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': 0.02,
             "boosting": "gbdt",
             "feature_fraction": 0.7522,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "metric": 'rmse',
             "lambda_l1": 0.2634,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param_ = folds_dict['param'].iloc[0]
print(best_param_)

Fold  1 RMSE : 3.724464
Fold  2 RMSE : 3.581248
Fold  3 RMSE : 3.728943
Fold  4 RMSE : 3.606215
Fold  5 RMSE : 3.614128
Average RMSE = 3.65153936909821
80 [3.72446, 3.58125, 3.72894, 3.60621, 3.61413]
Fold  1 RMSE : 3.721694
Fold  2 RMSE : 3.582594
Fold  3 RMSE : 3.729251
Fold  4 RMSE : 3.606088
Fold  5 RMSE : 3.613948
Average RMSE = 3.6512406808951083
85 [3.72169, 3.58259, 3.72925, 3.60609, 3.61395]
Fold  1 RMSE : 3.722634
Fold  2 RMSE : 3.580897
Fold  3 RMSE : 3.728468
Fold  4 RMSE : 3.604353
Fold  5 RMSE : 3.612490
Average RMSE = 3.650308148635342
90 [3.72263, 3.5809, 3.72847, 3.60435, 3.61249]
Fold  1 RMSE : 3.723848
Fold  2 RMSE : 3.583368
Fold  3 RMSE : 3.727349
Fold  4 RMSE : 3.605712
Fold  5 RMSE : 3.613328
Average RMSE = 3.6512460586771693
95 [3.72385, 3.58337, 3.72735, 3.60571, 3.61333]
Fold  1 RMSE : 3.720258
Fold  2 RMSE : 3.580190
Fold  3 RMSE : 3.727734
Fold  4 RMSE : 3.605076
Fold  5 RMSE : 3.612455
Average RMSE = 3.649670772457701
100 [3.72026, 3.58019, 3.72773, 3.60508

In [19]:
%%time
#learning_rate
folds_dict = {}
for i in [0.001,0.002,0.003,0.004,0.005,0.006]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': best_param_,
             'min_data_in_leaf': 149, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': changed_param,
             "boosting": "gbdt",
             "feature_fraction": 0.7522,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "metric": 'rmse',
             "lambda_l1": 0.2634,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param__ = folds_dict['param'].iloc[0]
print(best_param__)

Fold  1 RMSE : 3.720423
Fold  2 RMSE : 3.579201
Fold  3 RMSE : 3.724570
Fold  4 RMSE : 3.601263
Fold  5 RMSE : 3.613338
Average RMSE = 3.648286055741715
0.001 [3.72042, 3.5792, 3.72457, 3.60126, 3.61334]
Fold  1 RMSE : 3.719945
Fold  2 RMSE : 3.578735
Fold  3 RMSE : 3.723997
Fold  4 RMSE : 3.601073
Fold  5 RMSE : 3.613398
Average RMSE = 3.647954512806694
0.002 [3.71995, 3.57873, 3.724, 3.60107, 3.6134]
Fold  1 RMSE : 3.719573
Fold  2 RMSE : 3.578329
Fold  3 RMSE : 3.725185
Fold  4 RMSE : 3.600795
Fold  5 RMSE : 3.613536
Average RMSE = 3.6480140132036736
0.003 [3.71957, 3.57833, 3.72519, 3.60079, 3.61354]
Fold  1 RMSE : 3.719127
Fold  2 RMSE : 3.578785
Fold  3 RMSE : 3.724865
Fold  4 RMSE : 3.601450
Fold  5 RMSE : 3.612926
Average RMSE = 3.647955932966454
0.004 [3.71913, 3.57879, 3.72487, 3.60145, 3.61293]
Fold  1 RMSE : 3.718847
Fold  2 RMSE : 3.578153
Fold  3 RMSE : 3.725141
Fold  4 RMSE : 3.602227
Fold  5 RMSE : 3.613218
Average RMSE = 3.648042245661085
0.005 [3.71885, 3.57815, 3.725

In [20]:
%%time
#lambda_l1
folds_dict = {}
for i in [0.1,0.15,0.2,0.25,0.3,0.35]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': best_param_,
             'min_data_in_leaf': 149, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': best_param__,
             "boosting": "gbdt",
             "feature_fraction": 0.7522,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "metric": 'rmse',
             "lambda_l1": changed_param,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param2 = folds_dict['param'].iloc[0]
print(best_param2)

Fold  1 RMSE : 3.719969
Fold  2 RMSE : 3.578675
Fold  3 RMSE : 3.724166
Fold  4 RMSE : 3.600987
Fold  5 RMSE : 3.613223
Average RMSE = 3.6479305072657566
0.1 [3.71997, 3.57868, 3.72417, 3.60099, 3.61322]
Fold  1 RMSE : 3.720041
Fold  2 RMSE : 3.578683
Fold  3 RMSE : 3.724155
Fold  4 RMSE : 3.601024
Fold  5 RMSE : 3.613131
Average RMSE = 3.647933457709131
0.15 [3.72004, 3.57868, 3.72415, 3.60102, 3.61313]
Fold  1 RMSE : 3.720297
Fold  2 RMSE : 3.578962
Fold  3 RMSE : 3.724127
Fold  4 RMSE : 3.601127
Fold  5 RMSE : 3.613534
Average RMSE = 3.648135181330363
0.2 [3.7203, 3.57896, 3.72413, 3.60113, 3.61353]
Fold  1 RMSE : 3.719664
Fold  2 RMSE : 3.578894
Fold  3 RMSE : 3.724167
Fold  4 RMSE : 3.601062
Fold  5 RMSE : 3.613112
Average RMSE = 3.6479042340070515
0.25 [3.71966, 3.57889, 3.72417, 3.60106, 3.61311]
Fold  1 RMSE : 3.719987
Fold  2 RMSE : 3.578799
Fold  3 RMSE : 3.723961
Fold  4 RMSE : 3.601119
Fold  5 RMSE : 3.613242
Average RMSE = 3.6479465845560823
0.3 [3.71999, 3.5788, 3.72396, 

In [21]:
%%time
#feature_fraction
folds_dict = {}
for i in [0.6,0.65,0.7,0.75,0.80,0.85]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': best_param_,
             'min_data_in_leaf': 149, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': best_param__,
             "boosting": "gbdt",
             "feature_fraction": changed_param,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "metric": 'rmse',
             "lambda_l1": best_param2,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param3 = folds_dict['param'].iloc[0]
print(best_param3)

Fold  1 RMSE : 3.719580
Fold  2 RMSE : 3.578338
Fold  3 RMSE : 3.723862
Fold  4 RMSE : 3.600322
Fold  5 RMSE : 3.613039
Average RMSE = 3.6475550649294015
0.6 [3.71958, 3.57834, 3.72386, 3.60032, 3.61304]
Fold  1 RMSE : 3.720999
Fold  2 RMSE : 3.578821
Fold  3 RMSE : 3.723781
Fold  4 RMSE : 3.600944
Fold  5 RMSE : 3.613868
Average RMSE = 3.6482098819845343
0.65 [3.721, 3.57882, 3.72378, 3.60094, 3.61387]
Fold  1 RMSE : 3.720016
Fold  2 RMSE : 3.579102
Fold  3 RMSE : 3.724031
Fold  4 RMSE : 3.601189
Fold  5 RMSE : 3.614299
Average RMSE = 3.6482495194179423
0.7 [3.72002, 3.5791, 3.72403, 3.60119, 3.6143]
Fold  1 RMSE : 3.720225
Fold  2 RMSE : 3.578894
Fold  3 RMSE : 3.723950
Fold  4 RMSE : 3.601067
Fold  5 RMSE : 3.613349
Average RMSE = 3.64802228578068
0.75 [3.72022, 3.57889, 3.72395, 3.60107, 3.61335]
Fold  1 RMSE : 3.720203
Fold  2 RMSE : 3.578565
Fold  3 RMSE : 3.724753
Fold  4 RMSE : 3.601860
Fold  5 RMSE : 3.612671
Average RMSE = 3.6481395909142225
0.8 [3.7202, 3.57856, 3.72475, 3.6

In [22]:
%%time
#min_data_in_leaf
folds_dict = {}
for i in [100,125,150,175,200]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': best_param_,
             'min_data_in_leaf': changed_param, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': best_param__,
             "boosting": "gbdt",
             "feature_fraction": best_param3,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "metric": 'rmse',
             "lambda_l1": best_param2,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param4 = folds_dict['param'].iloc[0]
print(best_param4)

Fold  1 RMSE : 3.720022
Fold  2 RMSE : 3.579567
Fold  3 RMSE : 3.724586
Fold  4 RMSE : 3.599407
Fold  5 RMSE : 3.614053
Average RMSE = 3.648054482668975
100 [3.72002, 3.57957, 3.72459, 3.59941, 3.61405]
Fold  1 RMSE : 3.719855
Fold  2 RMSE : 3.578782
Fold  3 RMSE : 3.724566
Fold  4 RMSE : 3.599315
Fold  5 RMSE : 3.613564
Average RMSE = 3.6477474399971825
125 [3.71986, 3.57878, 3.72457, 3.59932, 3.61356]
Fold  1 RMSE : 3.720032
Fold  2 RMSE : 3.578654
Fold  3 RMSE : 3.724074
Fold  4 RMSE : 3.600625
Fold  5 RMSE : 3.612909
Average RMSE = 3.647786795514415
150 [3.72003, 3.57865, 3.72407, 3.60063, 3.61291]
Fold  1 RMSE : 3.720563
Fold  2 RMSE : 3.578184
Fold  3 RMSE : 3.724519
Fold  4 RMSE : 3.600710
Fold  5 RMSE : 3.612695
Average RMSE = 3.647868254059641
175 [3.72056, 3.57818, 3.72452, 3.60071, 3.6127]
Fold  1 RMSE : 3.720410
Fold  2 RMSE : 3.578824
Fold  3 RMSE : 3.724501
Fold  4 RMSE : 3.600381
Fold  5 RMSE : 3.613015
Average RMSE = 3.6479574532981913
200 [3.72041, 3.57882, 3.7245, 3.6

In [23]:
%%time
#max_depth
folds_dict = {}
for i in [7,8,9,10,11]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': best_param_,
             'min_data_in_leaf': best_param4, 
             'objective':'regression',
             'max_depth': changed_param,
             'learning_rate': best_param__,
             "boosting": "gbdt",
             "feature_fraction": best_param3,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "metric": 'rmse',
             "lambda_l1": best_param2,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param5 = folds_dict['param'].iloc[0]
print(best_param5)

Fold  1 RMSE : 3.720991
Fold  2 RMSE : 3.578391
Fold  3 RMSE : 3.725780
Fold  4 RMSE : 3.598861
Fold  5 RMSE : 3.613974
Average RMSE = 3.6481420514878296
7 [3.72099, 3.57839, 3.72578, 3.59886, 3.61397]
Fold  1 RMSE : 3.719736
Fold  2 RMSE : 3.577822
Fold  3 RMSE : 3.725490
Fold  4 RMSE : 3.599011
Fold  5 RMSE : 3.613281
Average RMSE = 3.647607427842794
8 [3.71974, 3.57782, 3.72549, 3.59901, 3.61328]
Fold  1 RMSE : 3.719855
Fold  2 RMSE : 3.578782
Fold  3 RMSE : 3.724566
Fold  4 RMSE : 3.599315
Fold  5 RMSE : 3.613564
Average RMSE = 3.6477474399971825
9 [3.71986, 3.57878, 3.72457, 3.59932, 3.61356]
Fold  1 RMSE : 3.720190
Fold  2 RMSE : 3.579156
Fold  3 RMSE : 3.724505
Fold  4 RMSE : 3.599550
Fold  5 RMSE : 3.613899
Average RMSE = 3.6479897060418534
10 [3.72019, 3.57916, 3.72451, 3.59955, 3.6139]
Fold  1 RMSE : 3.720654
Fold  2 RMSE : 3.579066
Fold  3 RMSE : 3.724512
Fold  4 RMSE : 3.600181
Fold  5 RMSE : 3.613939
Average RMSE = 3.6482006698791722
11 [3.72065, 3.57907, 3.72451, 3.60018,

In [24]:
%%time
#bagging_fraction
folds_dict = {}
for i in [0.5,0.6,0.7,0.8,0.9]:
    changed_param = i
    folds_list = []   
    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
        train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
        valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

        # set data structure
        lgb_train = lgb.Dataset(train_x,
                                label=train_y,
                                free_raw_data=False)
        lgb_test = lgb.Dataset(valid_x,
                               label=valid_y,
                               free_raw_data=False)

        # params optimized by optuna
        params = {'num_leaves': best_param_,
             'min_data_in_leaf': best_param4, 
             'objective':'regression',
             'max_depth': best_param5,
             'learning_rate': best_param__,
             "boosting": "gbdt",
             "feature_fraction": best_param3,
             "bagging_freq": 1,
             "bagging_fraction": changed_param ,
             "metric": 'rmse',
             "lambda_l1": best_param2,
             "random_state": SEED,
             "verbosity": -1}

        reg = lgb.train(
                        params,
                        lgb_train,
                        valid_sets=[lgb_train, lgb_test],
                        valid_names=['train', 'test'],
                        num_boost_round=10000,
                        early_stopping_rounds= 200,
                        verbose_eval=False
                        )

        oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
        sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        fold_rmse = rmse(valid_y, oof_preds[valid_idx])
        folds_list.append(round(fold_rmse,5))
        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()
    print("Average RMSE = {}".format(rmse(y,oof_preds)))
    folds_dict[str(i)] = folds_list
    print(i,folds_list)
folds_dict = pd.Series(folds_dict).reset_index(drop=False)
folds_dict.columns = ['param','value_list']
folds_dict['mean_value'] = folds_dict['value_list'].apply(lambda x: np.mean(x))
folds_dict = folds_dict.sort_values(by = 'mean_value',ascending=True)
best_param6 = folds_dict['param'].iloc[0]
print(best_param6)

Fold  1 RMSE : 3.722243
Fold  2 RMSE : 3.578370
Fold  3 RMSE : 3.725117
Fold  4 RMSE : 3.603511
Fold  5 RMSE : 3.614095
Average RMSE = 3.649199696471438
0.5 [3.72224, 3.57837, 3.72512, 3.60351, 3.6141]
Fold  1 RMSE : 3.721397
Fold  2 RMSE : 3.578618
Fold  3 RMSE : 3.725196
Fold  4 RMSE : 3.600746
Fold  5 RMSE : 3.615057
Average RMSE = 3.6487366438867936
0.6 [3.7214, 3.57862, 3.7252, 3.60075, 3.61506]
Fold  1 RMSE : 3.720128
Fold  2 RMSE : 3.577702
Fold  3 RMSE : 3.724851
Fold  4 RMSE : 3.599761
Fold  5 RMSE : 3.613130
Average RMSE = 3.6476512899593736
0.7 [3.72013, 3.5777, 3.72485, 3.59976, 3.61313]
Fold  1 RMSE : 3.720334
Fold  2 RMSE : 3.579538
Fold  3 RMSE : 3.724010
Fold  4 RMSE : 3.598407
Fold  5 RMSE : 3.612696
Average RMSE = 3.6475289545802445
0.8 [3.72033, 3.57954, 3.72401, 3.59841, 3.6127]
Fold  1 RMSE : 3.718624
Fold  2 RMSE : 3.579197
Fold  3 RMSE : 3.724272
Fold  4 RMSE : 3.599300
Fold  5 RMSE : 3.613590
Average RMSE = 3.6475200807947528
0.9 [3.71862, 3.5792, 3.72427, 3.599

In [25]:
{'num_leaves': best_param_,
 'min_data_in_leaf': best_param4, 
 'objective':'regression',
 'max_depth': best_param5,
 'learning_rate': best_param__,
 "boosting": "gbdt",
 "feature_fraction": best_param3,
 "bagging_freq": 1,
 "bagging_fraction": best_param6 ,
 "metric": 'rmse',
 "lambda_l1": best_param2,
 "random_state": SEED,
 "verbosity": -1}

{'num_leaves': '100',
 'min_data_in_leaf': '125',
 'objective': 'regression',
 'max_depth': '8',
 'learning_rate': '0.002',
 'boosting': 'gbdt',
 'feature_fraction': '0.6',
 'bagging_freq': 1,
 'bagging_fraction': '0.9',
 'metric': 'rmse',
 'lambda_l1': '0.25',
 'random_state': 51,
 'verbosity': -1}